In [12]:
tau = 100
C = 1000
feature_number = 20
test_size = 0.9
y_feature = 'Class'

In [13]:
import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
data = pd.read_csv('row_data-20231124T151822Z-001/row_data/data20F.csv')[840:940]
features = data.drop(columns=y_feature,axis=1)
labels = data[y_feature]
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V12,V13,V14,V15,V16,V17,V18,V19,V20,Class
840,1.134880,-0.214241,0.528744,1.328837,-0.426471,0.269760,-0.294896,0.095271,1.057808,-0.274677,...,0.093055,-0.486427,-0.367803,-0.800325,-0.297493,-0.102663,-0.093876,0.461431,-0.073023,0
841,-2.226521,1.361693,2.025260,0.823238,0.244459,0.599506,2.650073,-2.446798,2.269755,3.621816,...,-1.139200,-1.477087,-2.146990,0.805907,-2.074258,-0.165043,-1.047570,0.751404,-0.005224,0
842,-3.055529,-0.557340,2.144689,3.289656,0.273454,1.473169,-0.423035,0.765013,-0.421320,1.196380,...,0.913012,0.274182,-0.461024,-0.125364,-0.670081,0.726382,-0.530679,0.917471,-0.325894,0
843,-2.829522,-3.027379,1.260681,-0.467058,0.192918,-1.291539,1.646201,-0.475091,-1.909817,-0.218002,...,0.151756,1.412842,-0.164931,0.228741,-1.193028,-0.461535,0.674943,-1.284988,1.434089,0
844,-1.740943,0.328033,1.351311,-1.843991,-0.993856,0.377589,-1.153172,1.259231,-1.504410,-0.192535,...,0.666775,0.954289,0.085076,0.149902,1.445272,0.404312,-1.194503,-0.787398,-0.006535,0


In [28]:
labels

840    0
841    0
842    0
843    0
844    0
      ..
935    0
936    0
937    0
938    0
939    0
Name: Class, Length: 100, dtype: int64

In [16]:
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.9, shuffle=False
)

In [17]:
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=feature_number, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [18]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

In [19]:
start_time = time.time()
pegasos_qsvc.fit(train_features, train_labels)
elapsed_time = time.time() - start_time
print(f'Time spent to fit: {elapsed_time}' )

ValueError: Only binary classification is supported

In [ ]:
start_time = time.time()
predict_label = pegasos_qsvc.predict(test_features)
elapsed_time = time.time() - start_time
elapsed_time
print(f'Time spent to predict: {elapsed_time}' )

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(test_labels,predict_label))

In [ ]:
print(classification_report(test_labels,predict_label))

In [ ]:
start_time = time.time()
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
elapsed_time = time.time() - start_time
print(f'score : {pegasos_score} \ntime to score: {elapsed_time}' )